In [27]:
import pandas as pd
import numpy as np

def view_tests(initializations, algorithms, file_path, style=True):
    """
    Perform tests on the initialization and algorithm methods.
    
    Parameters
    ----------
    initializations : list
        List of initialization methods
    algorithms : list
        List of algorithm methods
    file_path : str
        Path of the file to load
    
    Returns
    -------
    pd.DataFrame
        Table with algorithms as columns, each column divided for initializations and rows as: algorithms, centroid index, time execution
    """

    data_frames = {}

    # read table from path
    name = ''.join(file_path.split('/')[1:]).split('.')[0]


    for algorithm in algorithms:
        test_data = pd.read_csv(f'tests/{name}-{algorithm}.csv', sep=',', engine='python')

        test_data = test_data[test_data["initialization"].isin(initializations)]
        data_frames[algorithm] = test_data.groupby("initialization", sort=False).agg(lambda x: list(x))

        # compute means/maxs/mins and count
        data_frames[algorithm]['count'] = data_frames[algorithm]['TIME_EXECUTION'].apply(lambda x: len(x))
        
        if not data_frames[algorithm]['CENTROID_INDEX'].isna().any():
            data_frames[algorithm]['min_centroid_index'] = data_frames[algorithm]['CENTROID_INDEX'].apply(lambda x: np.min(x))
            data_frames[algorithm]['max_centroid_index'] = data_frames[algorithm]['CENTROID_INDEX'].apply(lambda x: np.max(x))
            data_frames[algorithm]['mean_centroid_index'] = data_frames[algorithm]['CENTROID_INDEX'].apply(lambda x: np.mean(x))
            data_frames[algorithm]['std_centroid_index'] = data_frames[algorithm]['CENTROID_INDEX'].apply(lambda x: np.std(x))
        
        data_frames[algorithm]['mean_time_execution'] = data_frames[algorithm]['TIME_EXECUTION'].apply(lambda x: np.mean(x))
        
        data_frames[algorithm]['min_final_cost'] = data_frames[algorithm]['FINAL_COST'].apply(lambda x: np.min(x))
        data_frames[algorithm]['mean_final_cost'] = data_frames[algorithm]['FINAL_COST'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['max_final_cost'] = data_frames[algorithm]['FINAL_COST'].apply(lambda x: np.max(x))
        data_frames[algorithm]['std_final_cost'] = data_frames[algorithm]['FINAL_COST'].apply(lambda x: np.std(x))
        
        data_frames[algorithm]['mean_safe_iterations'] = data_frames[algorithm]['SAFE_ITERATIONS'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['mean_unsafe_iterations'] = data_frames[algorithm]['UNSAFE_ITERATIONS'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['mean_lloyd_iterations'] = data_frames[algorithm]['LLYOID_ITERATIONS'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['mean_hartigan_iterations'] = data_frames[algorithm]['HARTIGAN_ITERATIONS'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['mean_binary_iterations'] = data_frames[algorithm]['BINARY_ITERATIONS'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['mean_tot_iterations'] = data_frames[algorithm]['SAFE_ITERATIONS'].apply(lambda x: np.mean(x)) + data_frames[algorithm]['UNSAFE_ITERATIONS'].apply(lambda x: np.mean(x)) + data_frames[algorithm]['LLYOID_ITERATIONS'].apply(lambda x: np.mean(x)) + data_frames[algorithm]['HARTIGAN_ITERATIONS'].apply(lambda x: np.mean(x)) + data_frames[algorithm]['BINARY_ITERATIONS'].apply(lambda x: np.mean(x))
        
        data_frames[algorithm]['mean_init_norm_calculations'] = data_frames[algorithm]['INIT_NORM_CALCULATIONS'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['mean_norm_calculations'] = data_frames[algorithm]['NORM_CALCULATIONS'].apply(lambda x: np.mean(x))
        data_frames[algorithm]['mean_tot_norm_calculations'] = data_frames[algorithm]['INIT_NORM_CALCULATIONS'].apply(lambda x: np.mean(x)) + data_frames[algorithm]['NORM_CALCULATIONS'].apply(lambda x: np.mean(x))

        # drop columns
        data_frames[algorithm].drop(columns=['CENTROID_INDEX', 'TIME_EXECUTION', 'FINAL_COST', 'SAFE_ITERATIONS', 'UNSAFE_ITERATIONS', 'LLYOID_ITERATIONS', 'HARTIGAN_ITERATIONS', 'BINARY_ITERATIONS', 'INIT_NORM_CALCULATIONS', 'NORM_CALCULATIONS'], inplace=True)

    result_df = pd.concat(data_frames, axis=0, keys=algorithms, names=["Algorithm", "Initialization"])
    
    def color_specific_columns(x):
        df_styled = pd.DataFrame('', index=x.index, columns=x.columns)
        df_styled.loc[:, ['min_final_cost', 'mean_tot_iterations', 'mean_tot_norm_calculations', 'mean_centroid_index']] = 'font-weight: bold'
        return df_styled

    def add_vertical_lines(x):
        df_styled = pd.DataFrame('', index=x.index, columns=x.columns)
        # Add borders after columns B and C
        df_styled.loc[:, ['count', 'min_centroid_index', 'mean_time_execution', 'min_final_cost', 'mean_safe_iterations', 'mean_init_norm_calculations']] = 'border-left: 1px solid gray'
        return df_styled

    if style:
        # Apply the styles to the DataFrame
        result_df = result_df.style.apply(color_specific_columns, axis=None).apply(add_vertical_lines, axis=None)

    return result_df

def min_found(initializations, algorithms, file_path, treshold):
    """
    Returns the percentage of times a minimum (under a certain treshold) was found for each initialization and algorithm.


    Parameters
    ----------
    initializations : list
        List of initialization methods
    algorithms : list
        List of algorithm methods
    file_path : str
        Path of the file to load
    treshold : float
        Treshold for the minimum


    Returns
    -------
    pd.DataFrame
        Table with algorithms as columns, each column divided for initializations and rows as: algorithms, centroid index, time execution
    """
    # read table from path
    name = ''.join(file_path.split('/')[1:]).split('.')[0]

    dict_data = {}
    for algorithm in algorithms:
        data = pd.read_csv(f'tests/{name}-{algorithm}.csv', sep=',', engine='python')
        data = data[data["initialization"].isin(initializations)]
        data = data.groupby("initialization", sort=False).agg(lambda x: list(x))
        dict_data[algorithm] = data

    result_df = pd.DataFrame(columns=initializations, index=algorithms)

    for algorithm, data in dict_data.items():
        for initialization in initializations:
            # calculate the percentage of times a minimum was found
            min_found = np.sum(np.array(data['FINAL_COST'][initialization]) <= treshold) / len(data['FINAL_COST'][initialization])
            result_df.loc[algorithm, initialization] = min_found
    
    # all entries must have 3 numbers after the comma
    result_df = result_df.map(lambda x: f'{x:.3f}')
    
    return result_df

## A SETS

### A1

In [29]:
view_tests(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/A-Sets/a1.txt')

In [3]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/A-Sets/a1.txt', 12146257522.258896)

,maximin,k-means++,greedy-k-means++
lloyd,0.006,0.002,0.017
hartigan,0.012,0.004,0.045
extended-hartigan,0.018,0.003,0.029
mixed-hartigan,0.016,0.004,0.042
mixed-extended-hartigan,0.013,0.010,0.044


### A2

In [5]:
view_tests(['random', 'maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/A-Sets/a2.txt')

In [5]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/A-Sets/a2.txt', 20286736641.652190)

,maximin,k-means++,greedy-k-means++
lloyd,0.001,0.000,0.045
hartigan,0.008,0.002,0.156
extended-hartigan,0.006,0.003,0.147
mixed-hartigan,0.009,0.001,0.150
mixed-extended-hartigan,0.007,0.003,0.136


### A3

In [6]:
view_tests(['random', 'maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/A-Sets/a3.txt')

In [6]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/A-Sets/a3.txt', 164577200000)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.000,0.000
hartigan,0.001,0.000,0.002
extended-hartigan,0.000,0.000,0.000
mixed-hartigan,0.000,0.000,0.002
mixed-extended-hartigan,0.000,0.001,0.002


In [7]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/A-Sets/a3.txt', 164588000000)


,maximin,k-means++,greedy-k-means++
lloyd,0.012,0.003,0.009
hartigan,0.011,0.003,0.006
extended-hartigan,0.015,0.001,0.005
mixed-hartigan,0.010,0.002,0.007
mixed-extended-hartigan,0.008,0.003,0.009


## BRIDGE

### $k = 5$

In [7]:
view_tests(['random', 'k-means++', 'maximin', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge5.txt')

In [8]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge5.txt', 37450552.1)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.000,0.000
hartigan,0.771,0.533,0.586
extended-hartigan,0.689,0.460,0.518
mixed-hartigan,0.710,0.493,0.530
mixed-extended-hartigan,0.695,0.481,0.504


In [9]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge5.txt', 37450555.1)

,maximin,k-means++,greedy-k-means++
lloyd,0.001,0.003,0.003
hartigan,0.771,0.533,0.586
extended-hartigan,0.689,0.460,0.518
mixed-hartigan,0.710,0.493,0.530
mixed-extended-hartigan,0.695,0.481,0.504


### $k=10$

In [8]:
view_tests(['random', 'k-means++', 'maximin', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge.txt')

In [10]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge.txt', 29225400)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.000,0.001
hartigan,0.002,0.061,0.045
extended-hartigan,0.008,0.058,0.045
mixed-hartigan,0.005,0.025,0.034
mixed-extended-hartigan,0.008,0.041,0.029


### $k = 100$

In [9]:
view_tests(['random', 'k-means++', 'maximin', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge100.txt')

In [11]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge100.txt', 14700000)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.000,0.000
hartigan,0.000,0.000,0.002
extended-hartigan,0.000,0.000,0.008
mixed-hartigan,0.000,0.000,0.000
mixed-extended-hartigan,0.000,0.000,0.007


In [12]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge100.txt', 14770000)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.000,0.000
hartigan,0.000,0.040,0.215
extended-hartigan,0.001,0.058,0.275
mixed-hartigan,0.000,0.020,0.210
mixed-extended-hartigan,0.000,0.041,0.244


In [13]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/bridge100.txt', 14910000)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.000,0.003
hartigan,0.238,0.782,0.984
extended-hartigan,0.523,0.779,0.992
mixed-hartigan,0.134,0.693,0.987
mixed-extended-hartigan,0.232,0.773,0.997


## HOUSE

### $k=50$

In [10]:
view_tests(['random', 'k-means++', 'maximin', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/housec5.txt')

In [14]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/housec5.txt', 370000)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.000,0.000
hartigan,0.125,0.071,0.149
extended-hartigan,0.108,0.107,0.165
mixed-hartigan,0.103,0.079,0.138
mixed-extended-hartigan,0.105,0.085,0.167


In [15]:
min_found(['maximin', 'k-means++', 'greedy-k-means++'], ['lloyd', 'hartigan', 'extended-hartigan', 'mixed-hartigan', 'mixed-extended-hartigan'], 'data/housec5.txt', 371000)

,maximin,k-means++,greedy-k-means++
lloyd,0.000,0.001,0.006
hartigan,0.207,0.152,0.291
extended-hartigan,0.224,0.182,0.298
mixed-hartigan,0.198,0.147,0.268
mixed-extended-hartigan,0.195,0.156,0.308
